In [1]:
import os, csv
import pandas as pd
import nibabel as nib
import ipywidgets
import matplotlib.pyplot as plt
import matplotlib as mpl
import nilearn.plotting
from IPython.display import display, clear_output
from ipywidgets import widgets, interact
import numpy as np
import regtricks as rt
from nibabel import nifti1
from PIL import Image
from nibabel.viewers import OrthoSlicer3D

%matplotlib notebook

D:\Softwares\Anaconda3\lib\site-packages\nilearn\datasets\__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}
def toggle_code(state):
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))
def button_action(value):
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
state = False
toggle_code(state)
button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
display(button)

ToggleButton(value=False, description='Show code')

In [3]:
def getMasked(mask, image):
    temp =np.zeros(image.shape)
    for id,x in np.ndenumerate(temp):
        if mask[id[0]][id[1]][id[2]] > 0:
            temp[id[0]][id[1]][id[2]] = image[id[0]][id[1]][id[2]]
        else:
            temp[id[0]][id[1]][id[2]] = np.nan
        
    new_image = nib.Nifti1Image(temp, affine=np.eye(4))
    return new_image

subjects_list = []
for i in range(1,12):
    subjects_list.append('sub-'+str(i))

In [4]:
#set the path here
subject_oxford_asl_path= ''
subject_oxasl_path= ''

PVC_options = [("No", 0),('Yes', 1)]
PVC_dropdown = ipywidgets.Dropdown(options=PVC_options,
                                value=0,
                                description="PVC:")

Subject_options = subjects_list
Subject_dropdown = ipywidgets.Dropdown(options=Subject_options,
                                value='sub-1',
                                description="Subject:")

calib_list = [('voxelwise(default)', 0), ("csf", 1), ("wm", 2)]
Calib_options = calib_list
Calib_dropdown = ipywidgets.Dropdown(options=Calib_options,
                                value=0,
                                description="Calibration method:")

def get_csv_by_index_oxasl(subject_name,pvc_index, calib_index):
    subject_path = os.path.join(os.getcwd(), 'oxasl_display', subject_name)
    calib_method = ['voxelwise', 'refregion']
    native_space_dir = os.path.join(subject_path, "oxasl_results", 'output', 'native')
    native_space_pvc_dir = os.path.join(subject_path, "oxasl_results", 'output_pvcorr', 'native')
    calib_folder = ['calib_voxelwise', 'calib_refregion_csf','calib_refregion_wm']
    region_analysis_dir = [] 
    region_analysis_pvc_dir = [] 
    for calib_f in calib_folder:
        region_analysis_dir.append(os.path.join(native_space_dir, calib_f))
        region_analysis_pvc_dir.append(os.path.join(native_space_pvc_dir, calib_f))
    

    perfusions_list = []
    pd_Cerebral_Ratio_list = []
    pd_GM_WM_Ratio_list = []
    pd_LICA_RICA_Ratio_list = []
    for idx in range(len(region_analysis_dir)):
        perfusion_df = pd.read_csv(os.path.join(region_analysis_dir[idx], 'roi_stats.csv'))
        perfusion_gm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_gm.csv'))
        perfusion_wm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_wm.csv'))
    
        nonPVC = pd.concat([perfusion_df.iloc[0,[0,2]].to_frame().T, perfusion_df.iloc[2,[0,2]].to_frame().T, perfusion_df.iloc[1,[0,2]].to_frame().T, perfusion_df.iloc[3,[0,2]].to_frame().T,perfusion_df.iloc[4,[0,2]].to_frame().T,perfusion_df.iloc[5,[0,2]].to_frame().T,perfusion_df.iloc[24,[0,2]].to_frame().T,perfusion_df.iloc[25,[0,2]].to_frame().T,perfusion_df.iloc[26,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
        PVC = pd.concat([perfusion_gm_df.iloc[0,[0,2]].to_frame().T, perfusion_gm_df.iloc[2,[0,2]].to_frame().T,perfusion_wm_df.iloc[1,[0,2]].to_frame().T,perfusion_wm_df.iloc[3,[0,2]].to_frame().T,perfusion_wm_df.iloc[4,[0,2]].to_frame().T,perfusion_wm_df.iloc[5,[0,2]].to_frame().T,perfusion_gm_df.iloc[24,[0,2]].to_frame().T,perfusion_gm_df.iloc[25,[0,2]].to_frame().T,perfusion_gm_df.iloc[26,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
        perfusions = [nonPVC, PVC]
        #for perf in perfusions:
        #    perf['SpCov'] = perf[['Std', 'Mean']].apply(lambda x:x['Std']*100/x['Mean'], axis = 1)
        perfusions_list.append(perfusions)
    
        LICA_RICA_Ratio_nonPVC = {'LICA':[perfusion_df.iloc[25,2]], 'RICA':[perfusion_df.iloc[24,2]], "L/R":[round(perfusion_df.iloc[25,2]/perfusion_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_nonPVC = pd.DataFrame(data=LICA_RICA_Ratio_nonPVC)
        LICA_RICA_Ratio_PVC = {'LICA':[perfusion_gm_df.iloc[25,2]], 'RICA':[perfusion_gm_df.iloc[24,2]], "L/R":[round(perfusion_gm_df.iloc[25,2]/perfusion_gm_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_PVC = pd.DataFrame(data=LICA_RICA_Ratio_PVC)


        pd_LICA_RICA_Ratio = [pd_LICA_RICA_Ratio_nonPVC, pd_LICA_RICA_Ratio_PVC]
        pd_LICA_RICA_Ratio_list.append(pd_LICA_RICA_Ratio)
        
        Cerebral_Ratio_nonPVC = {'Right_Cerebral':[perfusion_df.iloc[4,2]], 'Left_Cerebral':[perfusion_df.iloc[5,2]], "R/L":[round(perfusion_df.iloc[4,2]/perfusion_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_nonPVC = pd.DataFrame(data=Cerebral_Ratio_nonPVC)
        Cerebral_Ratio_Ratio_PVC = {'Right_Cerebral':[perfusion_wm_df.iloc[4,2]], 'Left_Cerebral':[perfusion_wm_df.iloc[5,2]], "R/L":[round(perfusion_wm_df.iloc[4,2]/perfusion_wm_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_PVC = pd.DataFrame(data=Cerebral_Ratio_Ratio_PVC)

        pd_Cerebral_Ratio = [pd_Cerebral_Ratio_Ratio_nonPVC, pd_Cerebral_Ratio_Ratio_PVC]
        pd_Cerebral_Ratio_list.append(pd_Cerebral_Ratio)
        
        GM_WM_Ratio_nonPVC = {'80%+GM':[perfusion_df.iloc[2,2]], '90%+WM':[perfusion_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_df.iloc[2,2]/perfusion_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_Ratio_nonPVC = pd.DataFrame(data=GM_WM_Ratio_nonPVC)
        GM_WM_Ratio_PVC = {'80%+GM':[perfusion_gm_df.iloc[2,2]], '90%+WM':[perfusion_wm_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_gm_df.iloc[2,2]/perfusion_wm_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_PVC = pd.DataFrame(data=GM_WM_Ratio_PVC)

        pd_GM_WM_Ratio = [pd_GM_WM_Ratio_Ratio_nonPVC, pd_GM_WM_Ratio_PVC]
        pd_GM_WM_Ratio_list.append(pd_GM_WM_Ratio)
    
    return [perfusions_list, pd_LICA_RICA_Ratio_list, pd_Cerebral_Ratio_list, pd_GM_WM_Ratio_list ]



def get_csv_by_index_oxford_asl(subject_name, pvc_index, calib_index):
    subject_path = os.path.join(os.getcwd(), 'oxford_asl_display', subject_name)
    calib_method = ['voxelwise', 'refregion']
    native_space_dir = os.path.join(os.path.join(subject_path, "oxford_asl_results"), 'native_space', calib_method[calib_index])
    region_analysis_dir = os.path.join(native_space_dir, 'region_analysis')
    
    perfusion_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis.csv'))
    perfusion_gm_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis_gm.csv'))
    perfusion_wm_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis_wm.csv'))
    
    nonPVC = pd.concat([perfusion_df.iloc[0,[0,2]].to_frame().T, perfusion_df.iloc[1,[0,2]].to_frame().T, perfusion_df.iloc[2,[0,2]].to_frame().T, perfusion_df.iloc[3,[0,2]].to_frame().T,perfusion_df.iloc[6,[0,2]].to_frame().T,perfusion_df.iloc[7,[0,2]].to_frame().T,perfusion_df.iloc[26,[0,2]].to_frame().T,perfusion_df.iloc[27,[0,2]].to_frame().T,perfusion_df.iloc[28,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
    PVC = pd.concat([perfusion_gm_df.iloc[0,[0,2]].to_frame().T,perfusion_gm_df.iloc[1,[0,2]].to_frame().T,perfusion_wm_df.iloc[2,[0,2]].to_frame().T, perfusion_wm_df.iloc[3,[0,2]].to_frame().T,perfusion_wm_df.iloc[6,[0,2]].to_frame().T,perfusion_wm_df.iloc[7,[0,2]].to_frame().T,perfusion_gm_df.iloc[26,[0,2]].to_frame().T,perfusion_gm_df.iloc[27,[0,2]].to_frame().T,perfusion_gm_df.iloc[28,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
    perfusions = [nonPVC, PVC]
    #for perf in perfusions:
    #    perf['SpCov'] = perf[['Std', 'Mean']].apply(lambda x:x['Std']*100/x['Mean'], axis = 1)
    
    LICA_RICA_Ratio_nonPVC = {'LICA':[perfusion_df.iloc[27,2]], 'RICA':[perfusion_df.iloc[26,2]], "L/R":[round(perfusion_df.iloc[27,2]/perfusion_df.iloc[26,2], 2)]}
    pd_LICA_RICA_Ratio_nonPVC = pd.DataFrame(data=LICA_RICA_Ratio_nonPVC)
    LICA_RICA_Ratio_PVC = {'LICA':[perfusion_gm_df.iloc[27,2]], 'RICA':[perfusion_gm_df.iloc[26,2]], "L/R":[round(perfusion_gm_df.iloc[27,2]/perfusion_gm_df.iloc[26,2], 2)]}
    pd_LICA_RICA_Ratio_PVC = pd.DataFrame(data=LICA_RICA_Ratio_PVC)
    
    pd_LICA_RICA_Ratio = [pd_LICA_RICA_Ratio_nonPVC, pd_LICA_RICA_Ratio_PVC]
    
    Cerebral_Ratio_nonPVC = {'Right_Cerebral_WM':[perfusion_df.iloc[6,2]], 'Left_Cerebral_WM':[perfusion_df.iloc[7,2]], "R/L":[round(perfusion_df.iloc[6,2]/perfusion_df.iloc[7,2], 2)]}
    pd_Cerebral_Ratio_Ratio_nonPVC = pd.DataFrame(data=Cerebral_Ratio_nonPVC)
    Cerebral_Ratio_Ratio_PVC = {'Right_Cerebral_WM':[perfusion_wm_df.iloc[6,2]], 'Left_Cerebral_WM':[perfusion_wm_df.iloc[7,2]], "R/L":[round(perfusion_wm_df.iloc[6,2]/perfusion_wm_df.iloc[7,2], 2)]}
    pd_Cerebral_Ratio_Ratio_PVC = pd.DataFrame(data=Cerebral_Ratio_Ratio_PVC)

    pd_Cerebral_Ratio = [pd_Cerebral_Ratio_Ratio_nonPVC, pd_Cerebral_Ratio_Ratio_PVC]
    
    GM_WM_Ratio_nonPVC = {'80%+GM':[perfusion_df.iloc[1,2]], '90%+WM':[perfusion_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_df.iloc[1,2]/perfusion_df.iloc[3,2], 2)]}
    pd_GM_WM_Ratio_Ratio_nonPVC = pd.DataFrame(data=GM_WM_Ratio_nonPVC)
    GM_WM_Ratio_PVC = {'80%+GM':[perfusion_gm_df.iloc[1,2]], '90%+WM':[perfusion_wm_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_gm_df.iloc[1,2]/perfusion_wm_df.iloc[3,2], 2)]}
    pd_GM_WM_Ratio_PVC = pd.DataFrame(data=GM_WM_Ratio_PVC)

    pd_GM_WM_Ratio = [pd_GM_WM_Ratio_Ratio_nonPVC, pd_GM_WM_Ratio_PVC]
    return [perfusions, pd_LICA_RICA_Ratio, pd_Cerebral_Ratio, pd_GM_WM_Ratio ]

    
def show_table(subject_name, pvc_index, calib_index):
    #subject_id = int(subject_name[4:])
    
        
    #try:
    if True:
        pd_list_oxasl= get_csv_by_index_oxasl(subject_name, pvc_index,calib_index)
        pd_list_oxford_asl= get_csv_by_index_oxford_asl(subject_name, pvc_index,calib_index)

        df_left = pd_list_oxasl[0][calib_index][pvc_index]
        df_right = pd_list_oxford_asl[0][pvc_index]
        df_left.columns = ['name', 'oxasl_mean']
        df_right.columns = ['name', 'oxford_asl_mean']

        df_final = pd.merge(df_left, df_right, on=['name'])
        df_final['difference'] = df_final.apply(lambda x:x['oxasl_mean'] - x['oxford_asl_mean'], axis=1)
        df_final['diff_percent'] = df_final.apply(lambda x:x['difference']/min(x['oxasl_mean'], x['oxford_asl_mean']), axis=1)
        df_final['diff_percent'] = round(df_final['diff_percent']*100,2)
        display(df_final)
        
        df_LICA_RICA_oxasl = pd_list_oxasl[1][calib_index][pvc_index].iloc[0,2]
        df_LICA_RICA_oxford_asl = pd_list_oxford_asl[1][pvc_index].iloc[0,2]
        #df_LICA_RICA_oxasl.columns = ['L/R_oxasl']
        #df_LICA_RICA_oxford_asl.columns = ['L/R_oxford_asl']
        df_LICA_RICA = pd.DataFrame([[df_LICA_RICA_oxasl, df_LICA_RICA_oxford_asl]], columns=['LICA/RICA_oxasl', 'LICA/RICA_oxford_asl'])
        
        #df_LICA_RICA = pd.merge(df_LICA_RICA_oxasl, df_LICA_RICA_oxford_asl)
        df_LICA_RICA['difference'] = df_LICA_RICA.apply(lambda x:x['LICA/RICA_oxasl'] - x['LICA/RICA_oxford_asl'], axis=1)
        display(df_LICA_RICA)
        
        
        df_RC_LC_oxasl = pd_list_oxasl[2][calib_index][pvc_index].iloc[0,2]
        df_RC_LC_oxford_asl = pd_list_oxford_asl[2][pvc_index].iloc[0,2]
        #df_LICA_RICA_oxasl.columns = ['L/R_oxasl']
        #df_LICA_RICA_oxford_asl.columns = ['L/R_oxford_asl']
        df_RC_LC = pd.DataFrame([[df_RC_LC_oxasl, df_RC_LC_oxford_asl]], columns=['Right_Cerebral/Left_Cerebral_oxasl', 'Right_Cerebral/Left_Cerebral_oxford_asl'])
        
        #df_LICA_RICA = pd.merge(df_LICA_RICA_oxasl, df_LICA_RICA_oxford_asl)
        df_RC_LC['difference'] = df_RC_LC.apply(lambda x:x['Right_Cerebral/Left_Cerebral_oxasl'] - x['Right_Cerebral/Left_Cerebral_oxford_asl'], axis=1)
        display(df_RC_LC)
        
        
        df_GM_WM_oxasl = pd_list_oxasl[3][calib_index][pvc_index].iloc[0,2]
        df_GM_WM_oxford_asl = pd_list_oxford_asl[3][pvc_index].iloc[0,2]
        #df_LICA_RICA_oxasl.columns = ['L/R_oxasl']
        #df_LICA_RICA_oxford_asl.columns = ['L/R_oxford_asl']
        df_GM_WM = pd.DataFrame([[df_GM_WM_oxasl, df_GM_WM_oxford_asl]], columns=['80%GM/90%WM_oxasl', '80%GM/90%WM_oxford_asl'])
        
        #df_LICA_RICA = pd.merge(df_LICA_RICA_oxasl, df_LICA_RICA_oxford_asl)
        df_GM_WM['difference'] = df_GM_WM.apply(lambda x:x['80%GM/90%WM_oxasl'] - x['80%GM/90%WM_oxford_asl'], axis=1)
        display(df_GM_WM)
    else:
    #except Exception as e:
        #print(e)
        pass
        #print("Region_analysis file doesn't exist.")

_ = interact(show_table,subject_name=Subject_dropdown, pvc_index=PVC_dropdown, calib_index=Calib_dropdown)



interactive(children=(Dropdown(description='Subject:', options=('sub-1', 'sub-2', 'sub-3', 'sub-4', 'sub-5', '…

In [5]:
PVC_options = [("No", 0),('Yes', 1)]
PVC_dropdown = ipywidgets.Dropdown(options=PVC_options,
                                value=0,
                                description="PVC:")

calib_list = [('voxelwise(default)', 0), ("csf", 1), ("wm", 2)]
Calib_options = calib_list
Calib_dropdown = ipywidgets.Dropdown(options=Calib_options,
                                value=0,
                                description="Calibration method:")

def get_csv_by_index_oxasl(subject_name,pvc_index, calib_index):
    subject_path = os.path.join(os.getcwd(), 'oxasl_display', subject_name)
    calib_method = ['voxelwise', 'refregion']
    native_space_dir = os.path.join(subject_path, "oxasl_results", 'output', 'native')
    native_space_pvc_dir = os.path.join(subject_path, "oxasl_results", 'output_pvcorr', 'native')
    calib_folder = ['calib_voxelwise', 'calib_refregion_csf','calib_refregion_wm']
    region_analysis_dir = [] 
    region_analysis_pvc_dir = [] 
    for calib_f in calib_folder:
        #print(os.path.join(native_space_dir, calib_f))
        region_analysis_dir.append(os.path.join(native_space_dir, calib_f))
        region_analysis_pvc_dir.append(os.path.join(native_space_pvc_dir, calib_f))
    

    perfusions_list = []
    pd_Cerebral_Ratio_list = []
    pd_GM_WM_Ratio_list = []
    pd_LICA_RICA_Ratio_list = []
    for idx in range(len(region_analysis_dir)):
        perfusion_df = pd.read_csv(os.path.join(region_analysis_dir[idx], 'roi_stats.csv'))
        perfusion_gm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_gm.csv'))
        perfusion_wm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_wm.csv'))
    
        nonPVC = pd.concat([perfusion_df.iloc[0,[0,2]].to_frame().T, perfusion_df.iloc[2,[0,2]].to_frame().T, perfusion_df.iloc[1,[0,2]].to_frame().T, perfusion_df.iloc[3,[0,2]].to_frame().T,perfusion_df.iloc[4,[0,2]].to_frame().T,perfusion_df.iloc[5,[0,2]].to_frame().T,perfusion_df.iloc[24,[0,2]].to_frame().T,perfusion_df.iloc[25,[0,2]].to_frame().T,perfusion_df.iloc[26,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
        PVC = pd.concat([perfusion_gm_df.iloc[0,[0,2]].to_frame().T, perfusion_gm_df.iloc[2,[0,2]].to_frame().T,perfusion_wm_df.iloc[1,[0,2]].to_frame().T,perfusion_wm_df.iloc[3,[0,2]].to_frame().T,perfusion_wm_df.iloc[4,[0,2]].to_frame().T,perfusion_wm_df.iloc[5,[0,2]].to_frame().T,perfusion_gm_df.iloc[24,[0,2]].to_frame().T,perfusion_gm_df.iloc[25,[0,2]].to_frame().T,perfusion_gm_df.iloc[26,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
        perfusions = [nonPVC, PVC]
        #for perf in perfusions:
        #    perf['SpCov'] = perf[['Std', 'Mean']].apply(lambda x:x['Std']*100/x['Mean'], axis = 1)
        perfusions_list.append(perfusions)
    
        LICA_RICA_Ratio_nonPVC = {'LICA':[perfusion_df.iloc[25,2]], 'RICA':[perfusion_df.iloc[24,2]], "L/R":[round(perfusion_df.iloc[25,2]/perfusion_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_nonPVC = pd.DataFrame(data=LICA_RICA_Ratio_nonPVC)
        LICA_RICA_Ratio_PVC = {'LICA':[perfusion_gm_df.iloc[25,2]], 'RICA':[perfusion_gm_df.iloc[24,2]], "L/R":[round(perfusion_gm_df.iloc[25,2]/perfusion_gm_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_PVC = pd.DataFrame(data=LICA_RICA_Ratio_PVC)


        pd_LICA_RICA_Ratio = [pd_LICA_RICA_Ratio_nonPVC, pd_LICA_RICA_Ratio_PVC]
        pd_LICA_RICA_Ratio_list.append(pd_LICA_RICA_Ratio)
        
        Cerebral_Ratio_nonPVC = {'Right_Cerebral':[perfusion_df.iloc[4,2]], 'Left_Cerebral':[perfusion_df.iloc[5,2]], "R/L":[round(perfusion_df.iloc[4,2]/perfusion_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_nonPVC = pd.DataFrame(data=Cerebral_Ratio_nonPVC)
        Cerebral_Ratio_Ratio_PVC = {'Right_Cerebral':[perfusion_wm_df.iloc[4,2]], 'Left_Cerebral':[perfusion_wm_df.iloc[5,2]], "R/L":[round(perfusion_wm_df.iloc[4,2]/perfusion_wm_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_PVC = pd.DataFrame(data=Cerebral_Ratio_Ratio_PVC)

        pd_Cerebral_Ratio = [pd_Cerebral_Ratio_Ratio_nonPVC, pd_Cerebral_Ratio_Ratio_PVC]
        pd_Cerebral_Ratio_list.append(pd_Cerebral_Ratio)
        
        GM_WM_Ratio_nonPVC = {'80%+GM':[perfusion_df.iloc[2,2]], '90%+WM':[perfusion_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_df.iloc[2,2]/perfusion_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_Ratio_nonPVC = pd.DataFrame(data=GM_WM_Ratio_nonPVC)
        GM_WM_Ratio_PVC = {'80%+GM':[perfusion_gm_df.iloc[2,2]], '90%+WM':[perfusion_wm_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_gm_df.iloc[2,2]/perfusion_wm_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_PVC = pd.DataFrame(data=GM_WM_Ratio_PVC)

        pd_GM_WM_Ratio = [pd_GM_WM_Ratio_Ratio_nonPVC, pd_GM_WM_Ratio_PVC]
        pd_GM_WM_Ratio_list.append(pd_GM_WM_Ratio)
    
    return [perfusions_list, pd_LICA_RICA_Ratio_list, pd_Cerebral_Ratio_list, pd_GM_WM_Ratio_list ]



def get_csv_by_index_oxford_asl(subject_name, pvc_index, calib_index):
    subject_path = os.path.join(os.getcwd(), 'oxford_asl_display', subject_name)
    calib_method = ['voxelwise', 'refregion']
    native_space_dir = os.path.join(os.path.join(subject_path, "oxford_asl_results"), 'native_space', calib_method[calib_index])
    region_analysis_dir = os.path.join(native_space_dir, 'region_analysis')
    
    perfusion_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis.csv'))
    perfusion_gm_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis_gm.csv'))
    perfusion_wm_df = pd.read_csv(os.path.join(region_analysis_dir, 'region_analysis_wm.csv'))
    
    nonPVC = pd.concat([perfusion_df.iloc[0,[0,2]].to_frame().T, perfusion_df.iloc[1,[0,2]].to_frame().T, perfusion_df.iloc[2,[0,2]].to_frame().T, perfusion_df.iloc[3,[0,2]].to_frame().T,perfusion_df.iloc[6,[0,2]].to_frame().T,perfusion_df.iloc[7,[0,2]].to_frame().T,perfusion_df.iloc[26,[0,2]].to_frame().T,perfusion_df.iloc[27,[0,2]].to_frame().T,perfusion_df.iloc[28,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
    PVC = pd.concat([perfusion_gm_df.iloc[0,[0,2]].to_frame().T,perfusion_gm_df.iloc[1,[0,2]].to_frame().T,perfusion_wm_df.iloc[2,[0,2]].to_frame().T, perfusion_wm_df.iloc[3,[0,2]].to_frame().T,perfusion_wm_df.iloc[6,[0,2]].to_frame().T,perfusion_wm_df.iloc[7,[0,2]].to_frame().T,perfusion_gm_df.iloc[26,[0,2]].to_frame().T,perfusion_gm_df.iloc[27,[0,2]].to_frame().T,perfusion_gm_df.iloc[28,[0,2]].to_frame().T], axis=0).reset_index(drop=True)
    perfusions = [nonPVC, PVC]
    #for perf in perfusions:
    #    perf['SpCov'] = perf[['Std', 'Mean']].apply(lambda x:x['Std']*100/x['Mean'], axis = 1)
    
    LICA_RICA_Ratio_nonPVC = {'LICA':[perfusion_df.iloc[27,2]], 'RICA':[perfusion_df.iloc[26,2]], "L/R":[round(perfusion_df.iloc[27,2]/perfusion_df.iloc[26,2], 2)]}
    pd_LICA_RICA_Ratio_nonPVC = pd.DataFrame(data=LICA_RICA_Ratio_nonPVC)
    LICA_RICA_Ratio_PVC = {'LICA':[perfusion_gm_df.iloc[27,2]], 'RICA':[perfusion_gm_df.iloc[26,2]], "L/R":[round(perfusion_gm_df.iloc[27,2]/perfusion_gm_df.iloc[26,2], 2)]}
    pd_LICA_RICA_Ratio_PVC = pd.DataFrame(data=LICA_RICA_Ratio_PVC)
    
    pd_LICA_RICA_Ratio = [pd_LICA_RICA_Ratio_nonPVC, pd_LICA_RICA_Ratio_PVC]
    
    Cerebral_Ratio_nonPVC = {'Right_Cerebral_WM':[perfusion_df.iloc[6,2]], 'Left_Cerebral_WM':[perfusion_df.iloc[7,2]], "R/L":[round(perfusion_df.iloc[6,2]/perfusion_df.iloc[7,2], 2)]}
    pd_Cerebral_Ratio_Ratio_nonPVC = pd.DataFrame(data=Cerebral_Ratio_nonPVC)
    Cerebral_Ratio_Ratio_PVC = {'Right_Cerebral_WM':[perfusion_wm_df.iloc[6,2]], 'Left_Cerebral_WM':[perfusion_wm_df.iloc[7,2]], "R/L":[round(perfusion_wm_df.iloc[6,2]/perfusion_wm_df.iloc[7,2], 2)]}
    pd_Cerebral_Ratio_Ratio_PVC = pd.DataFrame(data=Cerebral_Ratio_Ratio_PVC)

    pd_Cerebral_Ratio = [pd_Cerebral_Ratio_Ratio_nonPVC, pd_Cerebral_Ratio_Ratio_PVC]
    
    GM_WM_Ratio_nonPVC = {'80%+GM':[perfusion_df.iloc[1,2]], '90%+WM':[perfusion_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_df.iloc[1,2]/perfusion_df.iloc[3,2], 2)]}
    pd_GM_WM_Ratio_Ratio_nonPVC = pd.DataFrame(data=GM_WM_Ratio_nonPVC)
    GM_WM_Ratio_PVC = {'80%+GM':[perfusion_gm_df.iloc[1,2]], '90%+WM':[perfusion_wm_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_gm_df.iloc[1,2]/perfusion_wm_df.iloc[3,2], 2)]}
    pd_GM_WM_Ratio_PVC = pd.DataFrame(data=GM_WM_Ratio_PVC)

    pd_GM_WM_Ratio = [pd_GM_WM_Ratio_Ratio_nonPVC, pd_GM_WM_Ratio_PVC]
    return [perfusions, pd_LICA_RICA_Ratio, pd_Cerebral_Ratio, pd_GM_WM_Ratio ]


def get_results(pvc_index, calib_index):
    oxford_asl_results = []
    oxasl_results = []
    diff_results = []
    for i in range(1, 11):
        subject_name = 'sub-' + str(i)
        pd_list_oxasl= get_csv_by_index_oxasl(subject_name, pvc_index,calib_index)
        pd_list_oxford_asl= get_csv_by_index_oxford_asl(subject_name, pvc_index,calib_index)
        
        df_left = pd_list_oxasl[0][calib_index][pvc_index]
        df_right = pd_list_oxford_asl[0][pvc_index]
        df_left.columns = ['name', 'oxasl_mean']
        df_right.columns = ['name', 'oxford_asl_mean']

        df_final = pd.merge(df_left, df_right, on=['name'])
        df_final['difference'] = df_final.apply(lambda x:x['oxasl_mean'] - x['oxford_asl_mean'], axis=1)
        df_final['diff_percent'] = df_final.apply(lambda x:x['difference']/min(x['oxasl_mean'], x['oxford_asl_mean']), axis=1)
        df_final['diff_percent'] = round(df_final['diff_percent']*100, 2)
            
        #display(df_final)
        oxford_asl_results.append(df_final['oxford_asl_mean'].values.tolist())
        oxasl_results.append(df_final['oxasl_mean'].values.tolist())
        diff_results.append(df_final['diff_percent'].values.tolist())
        
        #print(oxford_asl_results)
    oxford_asl_results = np.array(oxford_asl_results).T
    oxasl_results = np.array(oxasl_results).T
    diff_results = np.array(diff_results).T
    return oxford_asl_results, oxasl_results, diff_results

def show_table(pvc_index, calib_index):
    #subject_id = int(subject_name[4:])
    
        
    try:
        oxford_asl_results = []
        oxasl_results = []
        diff_results = []
        '''
        for i in range(1, 11):
            subject_name = 'sub-' + str(i)
            pd_list_oxasl= get_csv_by_index_oxasl(subject_name, pvc_index,calib_index)
            pd_list_oxford_asl= get_csv_by_index_oxford_asl(subject_name, pvc_index,calib_index)
            
            df_left = pd_list_oxasl[0][calib_index][pvc_index]
            df_right = pd_list_oxford_asl[0][pvc_index]
            df_left.columns = ['name', 'oxasl_mean']
            df_right.columns = ['name', 'oxford_asl_mean']

            df_final = pd.merge(df_left, df_right, on=['name'])
            df_final['difference'] = df_final.apply(lambda x:x['oxasl_mean'] - x['oxford_asl_mean'], axis=1)
            df_final['diff_percent'] = df_final.apply(lambda x:x['difference']/min(x['oxasl_mean'], x['oxford_asl_mean']), axis=1)
            df_final['diff_percent'] = round(df_final['diff_percent']*100, 2)
            
            #display(df_final)
            oxford_asl_results.append(df_final['oxford_asl_mean'].values.tolist())
            oxasl_results.append(df_final['oxasl_mean'].values.tolist())
            diff_results.append(df_final['diff_percent'].values.tolist())
        
        #print(oxford_asl_results)
        oxford_asl_results = np.array(oxford_asl_results).T
        oxasl_results = np.array(oxasl_results).T
        #diff_results = oxasl_results - oxford_asl_results
        diff_results = np.array(diff_results).T
        #print(oxford_asl_results)
        #print(oxasl_results)
        #print(diff_results)
        '''
        oxford_asl_results,oxasl_results, diff_results = get_results(pvc_index, calib_index)
        oxford_asl_results2 = []
        oxasl_results2 = []
        diff_results2 = []
        
        oxford_asl_results2,oxasl_results2, diff_results2 = get_results(pvc_index, 1-calib_index)
        
        x = np.arange(1,11,1)
        fig=plt.figure(figsize=(8,13))
        ax= fig.add_subplot(3,1,1)
        #plt.plot(x, oxford_asl_results[0], 'r', label='10%GM')
        plt.plot(x, oxford_asl_results[1], 'g', label='80%GM')
        #plt.plot(x, oxford_asl_results[2], 'b', label='10%WM')
        plt.plot(x, oxford_asl_results[3], 'purple', label='80%WM')
        plt.plot(x, oxford_asl_results[4], 'c', label='RCWM')
        plt.plot(x, oxford_asl_results[5], 'gold', label='LCWM')
        plt.plot(x, oxford_asl_results[6], 'violet', label='RICA')
        plt.plot(x, oxford_asl_results[7], 'lime', label='LICA')
        plt.plot(x, oxford_asl_results[8], 'gray', label='VBA')
        plt.title("oxford_asl_results")
        plt.xlabel('Subject ID')
        plt.ylabel('Perfusion')
        plt.legend()
        
        ax= fig.add_subplot(3,1,2)
        #plt.plot(x, oxasl_results[0], 'r', label='10%GM')
        plt.plot(x, oxasl_results[1], 'g', label='80%GM')
        #plt.plot(x, oxasl_results[2], 'b', label='10%WM')
        plt.plot(x, oxasl_results[3], 'purple', label='80%WM')
        plt.plot(x, oxasl_results[4], 'c', label='RCWM')
        plt.plot(x, oxasl_results[5], 'gold', label='LCWM')
        plt.plot(x, oxasl_results[6], 'violet', label='RICA')
        plt.plot(x, oxasl_results[7], 'lime', label='LICA')
        plt.plot(x, oxasl_results[8], 'gray', label='VBA')
        plt.title("oxasl_results")
        plt.xlabel('Subject ID')
        plt.ylabel('Perfusion')
        plt.legend()
        
        ax= fig.add_subplot(3,1,3)
        #plt.plot(x, diff_results[0], 'r', label='10%GM')
        plt.plot(x, diff_results[1], 'g', label='80%GM')
        #plt.plot(x, diff_results[2], 'b', label='10%WM')
        plt.plot(x, diff_results[3], 'purple', label='80%WM')
        plt.plot(x, diff_results[4], 'c', label='RCWM')
        plt.plot(x, diff_results[5], 'gold', label='LCWM')
        plt.plot(x, diff_results[6], 'violet', label='RICA')
        plt.plot(x, diff_results[7], 'lime', label='LICA')
        plt.plot(x, diff_results[8], 'gray', label='VBA')
        plt.title("Difference of oxasl - oxford_asl")
        plt.xlabel('Subject ID')
        plt.ylabel('Difference percentage')
        plt.legend()
        
        plt.show()
    except Exception as e:
        print(e)
        #print("Region_analysis file doesn't exist.")

_ = interact(show_table, pvc_index=PVC_dropdown, calib_index=Calib_dropdown)



interactive(children=(Dropdown(description='PVC:', options=(('No', 0), ('Yes', 1)), value=0), Dropdown(descrip…